In [ ]:
import requests
import json
import csv

url = "https://www.sjvair.com/api/1.0/monitors/"
county_list = ['Stockton' ,'Manteca' , 'Modesto' , 'Delano', 'Bakersfield', 'Empire', 'Waterford', 'Ceres' ,'Turlock' ,'Grayson' ,'Patterson' ,'Merced' ,'Fairmead' ,'Madera', 'Clovis', 'Hanford' ,'Corcoran' ,'Visalia' ,'Terra' ,'Bella']
loop_county_list = [i for i in county_list]
r = requests.get(url , params = {"is_sjvair": "true" , "county":json.dumps(loop_county_list), "device":"PurpleAir"})
with open("data.json" , "w") as i:
    i.write(json.dumps(r.json() , indent = 4))

with open("data.json", "r") as read_file:
    data = json.load(read_file)

sensor_list = ["Clovis RAMP PA" , "Fresno CARB CCAC" , "Fresno CARB CCAC" , "CCAC_Office" , "ucm-88d4" , "Madison Out" , "Fresno-Pacific RAMP PA" , "Fresno-Foundry RAMP PA"
        , "ucm-8b6f" , "Hanford RAMP PA" , "Corcoran-Patterson RAMP PA" , "The Birds" , "Ms.Roosevelt" , "ucm-1c8a" , "Terra Bella Elementary School" , "Ducor Union Elementary School",
        "Albany Park School" , "Terrace Elementary School" , "Morningside Elementary School" , "Almond Tree Middle School" , "Fremont Elementary School" , "CCAC BAM - Delano" , "DELANO_Coloc",
        "Harvest and La Vina School" , "Pioneer School" ," Nueva Vista Language Academy" , "Del Vista Math and Science Academy" ," Princeton Street School" , "Bakersfield Golden RAMP PA" , 
        "Bakersfield CARB CCAC"
        ]

new_list = []

print(len(data["data"]))

for index in range(0  ,len(data['data'])):
    if data['data'][index]['name'] in sensor_list:
        new_list.append({
            'name': data['data'][index]['name'],
            'id': data['data'][index]['purple_id'],
            'lat': data['data'][index]['position']['coordinates'][1],
            'log': data['data'][index]['position']['coordinates'][0],
        })
        
with open("new_data.json" , "w") as i:
    i.write(json.dumps(new_list , indent = 4))



In [55]:
import pandas as pd
import time
import csv
import requests
import datetime
import json
import os

class Data_Downloading:
    
    def __init__(self , list_sensor , start_date , end_date , data , data_type): 
        self.list_sensor = list_sensor
        self.start_date = start_date
        self.end_date = end_date
        self.data = data
        self.data_type = data_type
        self.api_prupleair = {'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"}
        
    def get_data_json_svjair(self , name):
        
        response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{name}")
            
        sensor_data = response.json()
        
        result_list = {}
        
        if "name" in self.data:

            result_list = {
                'sensor_id': name,
                'name': sensor_data['data']['name'],
            }
        
        elif "log" in self.data or "lat" in self.data:
            result_list = {
                'sensor_id': name,
                'log': sensor_data['data']['position']['coordinates'][0],
                'lat': sensor_data['data']['position']['coordinates'][1],
            }
        
        return result_list
        
    def get_data_svjair(self):
        
        parameters = {'timestamp__gte': self.start_date, 'timestamp__lt': self.end_date}
        
        if not os.path.exists("svjair_folder"):
            os.makedirs("svjair_folder")

        for i in self.list_sensor:
            
            all_data = []
            
            csv_response = requests.get(f"https://www.sjvair.com/api/1.0/monitors/{i}/entries/csv/", params=parameters)
            
            with open(f"svjair_folder/SVJAir_{i}.csv", "wb") as f:
                f.write(csv_response.content)

            data_df = pd.read_csv(f"svjair_folder/SVJAir_{i}.csv")
            
            result_list = self.get_data_json_svjair(name=i)
            
            for j in range(len(data_df)):
                
                val = data_df.iloc[j]
                
                val_data = {}
                
                for k in self.data:
                    
                    try:
                        
                        val_data[k] = val[k]
                        
                    except KeyError:
                        continue
                    
                val_data.update(result_list)
                
                all_data.append(val_data)

            df_final = pd.DataFrame(all_data)
            
            df_final.to_csv(f"svjair_folder/SVJAir_{i}.csv", index=False)
            
            print(f" Finish Download svjair_folder/SVJAir_{i}.csv")
            
            
            
############################################################################


Data saved to purpleair_folder/sensor_155683_data.csv
Data saved to purpleair_folder/sensor_104106_data.csv


In [ ]:
class GetPurpleAirData:
    
    def __init__(self, sensor_list, start_date, end_date, headers, fields):
        self.sensor_list = sensor_list
        self.start_date = start_date
        self.end_date = end_date
        self.headers = headers
        self.fields = fields
        
    def get_data_purpleair(self):
        for index in range(len(self.sensor_list)):
            # Build the URL
            url = f'https://api.purpleair.com/v1/sensors/{self.sensor_list[index]}/history/csv?fields={",".join(self.fields)}'
            
            response = requests.get(url, headers=self.headers)
            
            if not os.path.exists("purpleair_folder"):
                os.makedirs("purpleair_folder")
                
            filename = f"purpleair_folder/sensor_{self.sensor_list[index]}_data.csv"
            
            with open(filename , "wb") as f:
                
                f.write(response.content)
                
            print(f"Data saved to {filename}")

In [ ]:
if __name__ == '__main__':
    
    data = Data_Downloading(list_sensor=["yiz5lVBeRf26NdX-KtrPlg" , "K6rfX7COTLKuQIFIh00gjA"] , start_date = "2023-11-13" , end_date= "2023-11-15" , data = ["name" , "celsius" , "pm25" , "humidity" , 'timestamp'] , data_type="SVJAIR")
    
    data.get_data_svjair()
    
    data = GetPurpleAirData(
        sensor_list=["155683", "104106"],
        start_date=datetime.datetime(2023, 11, 16,4,30,0),
        end_date=datetime.datetime(2023, 11, 16,7,0,0),
        headers={'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"},
        fields=["pm2.5_alt","pm2.5_atm","pressure","humidity"]
    )
    
    data.get_data_purpleair()